In [89]:
import heapq
import msgspec
import numpy
from typing import Any, List
import gc

In [132]:
class Record(msgspec.Struct):
    data: bytes = None
    def size(self):
        return len(self.data)

CHANK_MAX_SIZE = 10 * 1024
class Chank(msgspec.Struct):
    # sizes: numpy.ndarray = numpy.array([], dtype=numpy.int32)
    is_dirty: bool = False
    sizes: List[int] = []
    storage: List[Record] = []
    empty_nodes: List[int] = [] # пустые узлы, полученные в результате удаления
    free_space: int = CHANK_MAX_SIZE

    def __lt__(self, other: 'Chank') -> bool:
        return self.free_space < other.free_space
        
    def add(self, r: Record):
        self.is_dirty = True
        self.sizes.append(r.size())
        self.storage.append(r)
        self.calc_free_space()
        
    def calc_free_space(self):
        self.free_space = CHANK_MAX_SIZE - (len(self.sizes)*32 + sum(self.sizes) + 64)
        return self.free_space
        

In [133]:
r = Record("8176816632937481-97519863295166235061874bwjhbvjhs")

In [134]:
msgspec.json.encode(msgspec.msgpack.decode(msgspec.msgpack.encode(r))).decode()

'{"data":"8176816632937481-97519863295166235061874bwjhbvjhs"}'

In [135]:
r.size()

49

In [136]:
%%time

import string
_letters = string.ascii_letters + string.digits # + string.punctuation
def random_str() -> str:
    cnt = numpy.random.randint(0, len(_letters))
    return "".join([ _letters[i] for i in numpy.random.randint(0, len(_letters), cnt)])
    
    
chanks = []

for i in range(10000):
    chank: ch = Chank()
    for i in range(numpy.random.randint(300)):
        r = Record(random_str())
        if chank.free_space > (r.size() + 32 + 64):
            chank.add(r)
        else:
            break
    heapq.heappush(chanks, (chank.free_space, chank))

gc.collect()

CPU times: user 16.8 s, sys: 160 ms, total: 17 s
Wall time: 16.9 s


52488

In [137]:
[c for c in chanks if c.free_space < 0]

AttributeError: 'tuple' object has no attribute 'free_space'

In [155]:
ch = chanks[3910][1]

In [156]:
ch.free_space

576

In [61]:
import string
string.ascii_letters + string.digits + string.punctuation

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'